In [71]:
from bs4 import BeautifulSoup
import urllib.request
import re
import json
import pandas as pd

In [86]:
dummy_url = "http://memory.pvost.org/pages/dic.html"
page = urllib.request.urlopen(dummy_url)

In [87]:
#ищем все линки
soup = BeautifulSoup(page)
links = []
for item in soup("li", {"class": "main"}):
    for link in item.find_all("a"):
        if "href" in link.attrs:
            links.append(link["href"])

In [88]:
#собираем словарь с данными
biographies = []
for link in links: 
    page = "http://memory.pvost.org/pages/" + link
    try:
        page = urllib.request.urlopen(page)
    except Exception:
        continue
    soup_bio = BeautifulSoup(page)
    east_bio = {}
    info = soup_bio.find("p", {"class": "zag2"})
    biography = soup_bio.find("p", attrs={'class': None})
    biography = re.sub(r"\n\s{2,30}", "", biography.text)
    names = info.text
    names = re.sub(r'\([^()]*\)', "", names)
    try:
        dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
    except AttributeError:
        continue
    try:
        east_bio["surname"] = re.search(r"[А-Я]+-?[А-Я]+", names).group()
    except AttributeError:
        east_bio["surname"] = ""
    try: 
        birth = re.search(r"(\d{4}\??|\?)", dates).group()
    except AttributeError:
        continue
    try:
        east_bio["name"] = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names).group()
    except AttributeError:
        east_bio["name"] = ""
    try:
        name = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names).group()
    except AttributeError:
        continue
    try:
        east_bio["patronym"] = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names.split(name)[1]).group()
    except AttributeError:
        east_bio["patronym"] = ""
    try:
        east_bio["birth_year"] = birth
    except AttributeError:
        east_bio["birth_year"] = ""
    try:
        east_bio["death_year"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
    except AttributeError:
        east_bio["death_year"] = ""
    except IndexError:
        east_bio["death_year"] = ""
    try:
        east_bio["birth_place"] = re.search(r"[а-я]{1,3}\.{1}[А-Яа-я\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
    except IndexError:
        east_bio["birth_place"] = ""
    except AttributeError:
        east_bio["birth_place"] = ""
    biographies.append(east_bio)

In [89]:
#смотрим, что получилось в датафрейме
bio_parse = pd.DataFrame(biographies)

In [90]:
bio_parse

,surname,name,patronym,birth_year,death_year,birth_place
0,АБИХ,Рудольф,Петрович,1901,1940,г. Баку
1,АБРАМСКИЙ,Иехезкель,,1886,1976,м. Дашковичи Гродненской губ
2,АБРАМСОН,Мануил,Моисеевич,1898,1938,др. свед
3,АБУГОВ,Михаил,Захарович,1901,1941,зам. зав
4,АБУЗОВ,Степан,Семенович,1898,1938,с. Старые Бегучи Самарской губ
...,...,...,...,...,...,...
7317,ЧЕРНЫШЕВ,Евгений,Иванович,1894,,г. Симбирск
7318,ШИЙК,Андрей,Александрович,1891,1978,
7319,ЭЛИАСОВ,Лазарь,Ефимович,?,,
7320,ЯНЧЕВЕЦКИЙ,Дмитрий,Григорьевич,1889,1934,


In [81]:
#тестовый парсинг одной статьи
dummy_url_again = "http://memory.pvost.org/pages/abuzov.html"
page_east = urllib.request.urlopen(dummy_url_again)
soup_bio = BeautifulSoup(page_east)

In [85]:
east_bio = {}
info = soup_bio.find("p", {"class": "zag2"})
biography = soup_bio.find("p", attrs={'class': None})
biography = re.sub(r"\n\s{2,30}", "", biography.text)
names = info.text
names = re.sub(r'\([^()]*\)', "", names)
dates = re.search(r"((\d{4}\??|\?)-?(\d{4}\??|\?)?)", info.text).group()
east_bio["surname"] = re.search(r"[А-Я]+-?[А-Я]+", names).group()
east_bio["name"] = re.search(r"([А-Я]{1}[а-я]+)", names).group()
name = re.search(r"([А-Я]{1}[а-я]+)", names).group()
birth = re.search(r"(\d{4}\??|\?)", dates).group()

try:
    east_bio["patronym"] = re.search(r"([А-Я]{1}[а-я]+-?[А-Яа-я]+)", names.split(name)[1]).group()
except AttributeError:
    east_bio["patronym"] = ""
try:
#    east_bio["birth_year"] = re.search(r"1[8-9][0-9]{2}", info.text.split("-")[0]).group()
     east_bio["birth_year"] = birth
except AttributeError:
    east_bio["birth_year"] = ""
try:
    east_bio["death_year"] = re.search(r"1[8-9][0-9]{2}", dates.split(birth)[1]).group()
except IndexError:
    east_bio["death_year"] = ""
except AttributeError:
    east_bio["death_year"] = ""
try: 
    east_bio["birth_place"] = re.search(r"[а-я]{1,3}\.{1}[А-Яа-я\s]+(?=\.|,)", biography.split("Род. в ")[1]).group()
except IndexError:
    east_bio["birth_place"] = ""
print(east_bio)
print(birth)

{'surname': 'АБУЗОВ', 'name': 'Степан', 'patronym': 'Семенович', 'birth_year': '1898', 'death_year': '1938', 'birth_place': 'с. Старые Бегучи Самарской губ'}
1898


In [92]:
with open('vostokovedy_base.json', 'w', encoding='utf-8') as f:
    json.dump(biographies, f, ensure_ascii=False, indent=4)

In [91]:
biographies

[{'surname': 'АБИХ',
  'name': 'Рудольф',
  'patronym': 'Петрович',
  'birth_year': '1901',
  'death_year': '1940',
  'birth_place': 'г. Баку'},
 {'surname': 'АБРАМСКИЙ',
  'name': 'Иехезкель',
  'patronym': '',
  'birth_year': '1886',
  'death_year': '1976',
  'birth_place': 'м. Дашковичи Гродненской губ'},
 {'surname': 'АБРАМСОН',
  'name': 'Мануил',
  'patronym': 'Моисеевич',
  'birth_year': '1898',
  'death_year': '1938',
  'birth_place': 'др. свед'},
 {'surname': 'АБУГОВ',
  'name': 'Михаил',
  'patronym': 'Захарович',
  'birth_year': '1901',
  'death_year': '1941',
  'birth_place': 'зам. зав'},
 {'surname': 'АБУЗОВ',
  'name': 'Степан',
  'patronym': 'Семенович',
  'birth_year': '1898',
  'death_year': '1938',
  'birth_place': 'с. Старые Бегучи Самарской губ'},
 {'surname': 'АВЕРБУХ',
  'name': 'Владимир',
  'patronym': 'Борисович',
  'birth_year': '1889',
  'death_year': '1941',
  'birth_place': 'м. Березино Минской губ'},
 {'surname': 'АВДОЩЕНКОВ',
  'name': 'Амплий',
  'patron